In [4]:
import os
import time
import datetime
import numpy as np
import PySimpleGUI as sg 

In [5]:
def get_layout_user_item(record):
    Nama = record['Nama']
    NIM  = record['NIM']
    JenisKelamin = record['JenisKelamin']
    JamMasuk = record['JamMasuk']
    filename = record['NamaFoto']
    
    
    layout_photo = [[sg.Image(filename="photo/" + filename, key="foto-" + filename, size=(100,100))]]
    layout_user = [[sg.Text("Nama \t: " + Nama)],
                  [sg.Text("NIM \t: " + NIM)],
                  [sg.Text("Kelamin \t: " + JenisKelamin)],
                  [sg.Text("Masuk \t: " + JamMasuk)]]
    
    layout_item =[
                    sg.Column(layout_photo), 
                    sg.Column(layout_user)
                  ]
    return layout_item

In [35]:
def get_class_info():
    curr_date = datetime.datetime.now().strftime("%A, %d %B %Y")
    layout = [
        [sg.Text(curr_date, justification='right', size=(38,1), font=("Helvetica", 10, "italic"))],
        [sg.Text("Kelas \t\t: "), sg.InputText(key="Kelas", size=(24,1))],
        [sg.Text("Jumlah Peserta \t: "), sg.InputText(key="JumlahPeserta", size=(24,1))],
        [sg.Text("JumlahHadir \t: "), sg.Text("-", key="JumlahHadir", size=(24,1))],
        [sg.Text("JamMulai \t: "), sg.Text("-", key="JamMulai", size=(24,1))],
        [sg.Text("Durasi \t\t: "), sg.Text("-", key="Durasi", size=(24,1))],
        [sg.Text("Status \t\t: "), sg.Text("-", key="Status", size=(24,1))],
        [sg.Button("   Mulai   ", key="mulai"), sg.Button("   Akhiri   ", key="akhiri")]
    ]
    return layout

In [36]:
def get_layout_home(records):
    # list user     
    list_user_layout = [get_layout_user_item(record) for record in records]
    
    layout_left = [
        [
            sg.Column(list_user_layout, scrollable=True, vertical_scroll_only=True, size=(360, 400))
        ]
    ]
    
    
    # informasi kelas     
    layout_right = get_class_info()
    
    layout = [
        [
            sg.Column(layout_left),
            sg.VerticalSeparator(), # vertical separator
            sg.Column(layout_right)
        ]
    ]
    return layout

In [37]:
def get_layout(records):
    layout_home = get_layout_home(records)
    layout_history = [[sg.Text("Histori", size=(80,20))]]
    layout_user = [[sg.Text("User", size=(80,20))]]
    layout_pengaturan = [[sg.Text("Pengaturan", size=(80,20))]]

    layout = [[sg.TabGroup([
                            [sg.Tab('Home', layout_home), 
                             sg.Tab('History', layout_history),
                             sg.Tab('User', layout_user),
                             sg.Tab('Pengaturan', layout_pengaturan)]
                            ])
              ]]
    return layout

- main program

In [1]:
from user import User

In [2]:
user = User()

In [17]:
def create_window(layout):
    return sg.Window(title="Aplikasi Pencatatan Kehadiran", layout=layout, margins=(10, 10), finalize=True)

In [41]:
curr_date = datetime.datetime.now().strftime("%Y-%m-%d")

curr_user_records = []
layout = get_layout(curr_user_records)
window = create_window(layout)
window['akhiri'].update(disabled=True)

status = ''
start_time = ''

while True :
    event, values = window.read(timeout=25)
    if event == 'mulai' :
        window.close()
        
        start_time = datetime.datetime.now()
        curr_user_records = user.get_user_in_class(curr_date, values['Kelas'])
        layout = get_layout(curr_user_records)
        window = create_window(layout)
        
        status = 'berjalan'
        window['Kelas'].update(value=values['Kelas'])
        window['JumlahPeserta'].update(value=values['JumlahPeserta'])
        window['JumlahHadir'].update(value=len(curr_user_records))
        window['Status'].update(value=status)
        window['JamMulai'].update(value=start_time.strftime("%H:%M:%S"))
        
        window['mulai'].update(disabled=True)
        window['akhiri'].update(disabled=False)
        
    if event == 'akhiri':
        status = 'selesai'
        window['mulai'].update(disabled=False)
        window['akhiri'].update(disabled=True)
        
    if event == sg.WIN_CLOSED:
        break
        
    if status == 'berjalan':
        curr_time = datetime.datetime.now()
        diff_time = curr_time - start_time
        window['Durasi'].update(value=str(diff_time).split(".")[0])
        
window.close()